In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Set a random seed for reproducibility of splits and model training
RANDOM_STATE = 42

### Load train & test data
print("Loading data...")
try:
    df_train_orig = pd.read_csv("train.csv")
    df_test_orig = pd.read_csv("test.csv")
except FileNotFoundError:
    print("Ensure 'train.csv' and 'test.csv' are in the same directory as the script.")
    print("Using dummy data for demonstration purposes.")
    # Create dummy data if files are not found
    num_train_samples = 1000
    num_test_samples = 500
    num_features = 5
    df_train_orig = pd.DataFrame(np.random.rand(num_train_samples, num_features), columns=[f'feature_{i}' for i in range(num_features)])
    df_train_orig['Sex'] = np.random.choice(['male', 'female'], num_train_samples)
    df_train_orig['id'] = np.arange(num_train_samples)
    df_train_orig['Calories'] = np.random.rand(num_train_samples) * 500 + 50

    df_test_orig = pd.DataFrame(np.random.rand(num_test_samples, num_features), columns=[f'feature_{i}' for i in range(num_features)])
    df_test_orig['Sex'] = np.random.choice(['male', 'female'], num_test_samples)
    df_test_orig['id'] = np.arange(num_test_samples) + num_train_samples

Loading data...
Ensure 'train.csv' and 'test.csv' are in the same directory as the script.
Using dummy data for demonstration purposes.
